# IMPORT LIBRARIES

In [43]:
import torch
from torchvision import datasets
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import numpy

# LOAD TRAIN & TEST DATA

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

# DATA LOADERS

In [4]:
loaders = {
    'train' : DataLoader(train_data, 
                        batch_size=100, 
                        shuffle=True, 
                        num_workers=1),
    'test'  : DataLoader(test_data, 
                        batch_size=100, 
                        shuffle=True, 
                        num_workers=1), }

# THE CNN

In [5]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x
    

# LOSS FUNCTION & ACTIVATION FUNCTION

In [9]:
from torch import optim
cnn = CNN()
optimizer = optim.Adam(cnn.parameters(), lr = 0.01) 
loss_func = nn.CrossEntropyLoss()


# TRAIN THE NETWORK

In [14]:
from torch.autograd import Variable
num_epochs = 10
def train(num_epochs, cnn, loaders):
    cnn.train()
    total_step = len(loaders['train'])
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            optimizer.zero_grad()           
            loss.backward()
            optimizer.step()            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
train(num_epochs, cnn, loaders)

Epoch [1/10], Step [100/600], Loss: 0.0930
Epoch [1/10], Step [200/600], Loss: 0.1020
Epoch [1/10], Step [300/600], Loss: 0.0265
Epoch [1/10], Step [400/600], Loss: 0.1315
Epoch [1/10], Step [500/600], Loss: 0.0263
Epoch [1/10], Step [600/600], Loss: 0.0213
Epoch [2/10], Step [100/600], Loss: 0.0509
Epoch [2/10], Step [200/600], Loss: 0.0096
Epoch [2/10], Step [300/600], Loss: 0.0351
Epoch [2/10], Step [400/600], Loss: 0.0196
Epoch [2/10], Step [500/600], Loss: 0.0419
Epoch [2/10], Step [600/600], Loss: 0.3183
Epoch [3/10], Step [100/600], Loss: 0.0919
Epoch [3/10], Step [200/600], Loss: 0.0039
Epoch [3/10], Step [300/600], Loss: 0.0887
Epoch [3/10], Step [400/600], Loss: 0.0275
Epoch [3/10], Step [500/600], Loss: 0.0100
Epoch [3/10], Step [600/600], Loss: 0.0213
Epoch [4/10], Step [100/600], Loss: 0.0778
Epoch [4/10], Step [200/600], Loss: 0.0007
Epoch [4/10], Step [300/600], Loss: 0.0349
Epoch [4/10], Step [400/600], Loss: 0.0145
Epoch [4/10], Step [500/600], Loss: 0.0194
Epoch [4/10

# LET'S TEST

In [37]:
with torch.no_grad():
    correct = 0
    total = 0
    accuracy = 0
    for data in loaders['test']:
        images,labels = data
        test_output, last_layer = cnn(images)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        accuracy += (pred_y == labels).sum().item() / float(labels.size(0))
    print(accuracy/len(loaders['test']))

0.9858999999999999


In [39]:
def digit(th):
    return str(int(torch.argmax(model(images[th].view(-1, 784))[0])))

# PREDICTIONS

In [42]:
sample = next(iter(loaders['test']))
imgs, lbls = sample
actual_number = lbls[:10].numpy()
test_output, last_layer = cnn(imgs[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(f'Prediction number: {pred_y}')
print(f'Actual number: {actual_number}')

Prediction number: [1 1 9 3 2 5 1 9 7 1]
Actual number: [1 1 9 3 2 5 1 9 7 1]
